In [1]:
import numpy as np
import pandas as pd


ModuleNotFoundError: No module named 'numpy'

In [3]:
sammple_hist = np.load(r"C:\Users\tskor\Documents\data\histograms\60_buckets_9_bands_60_res\60\Accomack_51\2017\5-6.npy")


In [3]:
d = {'m5':5, 'm6':6}

print('unsorted: ', d, 'sorted: ',  sorted(d))

unsorted:  {'m5': 5, 'm6': 6} sorted:  ['m5', 'm6']


In [16]:
x = sammple_hist.reshape((9,-1))

In [17]:
for i in range(2):
    print(x[i,:])

[   0    0    0    0    0    0    0   14  638 2325 4123 6452 7469 8516
 6163 4186 2190 1048  641  421  377  220  124   94   58   49   43   62
   52   35   29   17    8   14    4    3    3    3    5    4    3    0
    0    2    0    5    5    4    7   11    4    9    4    2    3    2
    0    3    1    2]
[   0    0    0    0    0    0    0    0    0  596 4350 6808 7708 9471
 7134 4728 2060  879  590  378  170   92   44   59   51   46   29   53
   69   29   12    5    6    5    5    2    1    2    5    2    4    2
    2    4    2    3    5    8    8    7    9    2    2    4    0    0
    3    2    2    0]


In [1]:
from datasets import load_dataset

dataset = load_dataset("imdb")

print(dataset)


False

In [25]:
import numpy as np
import os
INPUT_PATH = r'C:\Users\tskor\Documents\data\histograms\histograms_county_year'
HISTOGRAM_NAME = 'histogram-31035-2017.npy'

In [26]:
data_to_investigate = np.load(os.path.join(INPUT_PATH, HISTOGRAM_NAME), allow_pickle=True)

#### Test multimodal LSTM

In [1]:
# Add this at the beginning of your notebook
%load_ext autoreload
%autoreload 2  
# Automatically reload all modules before executing code

In [22]:
import os
import sys
#  Add the root directory to the path
root_dir = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
sys.path.append(root_dir)
import tensorflow as tf
from tensorflow.keras.utils import plot_model
import numpy as np
from tensorflow.keras.optimizers import Adam

# Make sure eager execution is enabled
# tf.config.run_functions_eagerly(True)
tf.data.experimental.enable_debug_mode()
tf.keras.config.disable_traceback_filtering()

In [3]:
#Dataset
from data.test_train_split import test_train_split_multi_modal
from models.lstm import LstmWeather, PreprocessingHead, Embeddings

WEATHER_PATH = r"C:\Users\tskor\Documents\GitHub\inovation_project\4_Data_Sample\weather_data-adams-2017-CO-8001.csv"
HISTOGRAM_PATH = r"C:\Users\tskor\Documents\GitHub\inovation_project\4_Data_Sample\histogram-08001-2017_reduced.csv"
LABELS_PATH = r"C:\Users\tskor\Documents\GitHub\inovation_project\4_Data_Sample\combined_labels_with_fips.npy"

CAT_FEATURES = ["fips"]

In [30]:
datasets_monthly_train, datasets_monthly_test = test_train_split_multi_modal(
    WEATHER_PATH,
    HISTOGRAM_PATH,
    LABELS_PATH,
    index=["fips", "year"],
    months=[5, 7, 9],
)
histograms_train, weather_train, labels_train = datasets_monthly_train
idx = 0

histogram data shape: (6, 11)
weather data shape: (6, 15)
common_index:  MultiIndex([('01003', 2017),
            ('08001', 2017)],
           names=['fips', 'year'])


In [31]:
model = LstmWeather(weather_train,
histograms_train,
CAT_FEATURES)

In [32]:
optimizer = Adam(learning_rate=0.001)

In [33]:
model.compile(optimizer=optimizer, loss='mse')
model((weather_train, histograms_train))

shapes
processed_weather_data:  (2, 19)
processed_satellite_data:  (2, 15)
weather embeddings:  (2, 32)
satellite embeddings:  (2, 32)
weather sat length:  3
weather sat shape:  (2, 64)
lstm input shape:  (2, 3, 64)
lstm_output shape:  (2, 3, 1)
output shape:  (2, 3, 1)
shapes
processed_weather_data:  (2, 19)
processed_satellite_data:  (2, 15)
weather embeddings:  (2, 32)
satellite embeddings:  (2, 32)
weather sat length:  3
weather sat shape:  (2, 64)
lstm input shape:  (2, 3, 64)
lstm_output shape:  (2, 3, 1)
output shape:  (2, 3, 1)


<tf.Tensor: shape=(2, 3, 1), dtype=float32, numpy=
array([[[1.0980556],
        [1.0265042],
        [1.0750965]],

       [[1.0613073],
        [1.0763773],
        [1.1130291]]], dtype=float32)>

In [34]:
labels_train

array([[162.0],
       [82.0]], dtype=object)

In [35]:
model.fit(x=(weather_train, histograms_train), y=labels_train, epochs=2)

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type float).

In [29]:
preproc_histogram = PreprocessingHead(
    input_dict=histograms_train[0], cat_features=CAT_FEATURES
)
preproc_histogram(histograms_train[0])

<tf.Tensor: shape=(2, 15), dtype=float32, numpy=
array([[0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]],
      dtype=float32)>

In [23]:
model.summary()

Model: "lstm_weather_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ preprocessing_head_12           │ ?                      │   0 (unbuilt) │
│ (PreprocessingHead)             │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ preprocessing_head_13           │ ?                      │   0 (unbuilt) │
│ (PreprocessingHead)             │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ preprocessing_head_14           │ ?                      │   0 (unbuilt) │
│ (PreprocessingHead)             │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ preprocessing_head_15           │ ?                      │   0 (unbuilt) │
│ (PreprocessingHead)             │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ preprocessing_head_16           │ ?                      │   0 (unbuilt) │
│ (PreprocessingHead)             │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ preprocessing_head_17           │ ?                      │   0 (unbuilt) │
│ (PreprocessingHead)             │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embeddings_12 (Embeddings)      │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embeddings_13 (Embeddings)      │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embeddings_14 (Embeddings)      │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embeddings_15 (Embeddings)      │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embeddings_16 (Embeddings)      │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embeddings_17 (Embeddings)      │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ concatenate_42 (Concatenate)    │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ concatenate_43 (Concatenate)    │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ concatenate_44 (Concatenate)    │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_2 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_74 (Dense)                │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 174 (720.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 174 (720.00 B)